# os 모듈 사용법 - python/3.6 확인
-----

# 디렉토리 내에 폴더, 파일이 있는 지 먼저 확인
# 없으면 폴더를 만들어서 크롤링한 데이터를 저장
# 있으면 현재에 있는 데이터를 불러오거나 크롤링 데이터를 불러와서 덮어쓰기
# 반복해서 쓰이는 함수는 새로운 함수를 만들어서 계속해서 불러와서 사용

----------
# Ch5
# 업종분류 현황 크롤링 - 코스피

In [3]:
import requests
import pandas as pd
import csv

In [3]:
gen_stk_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
gen_stk_otp_data = {
    'mktId' : 'STK',   #### 코스피 = stk, 코스닥 = ksq -> option으로 넣으면 될듯
    'trdDd' : '20210312', #### 날짜 -> 추후 영업일로 사용! -> option으로 넣으면 될듯
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : "dbms/MDC/STAT/standard/MDCSTAT03901"
}

opt = requests.post(gen_stk_otp_url, data = gen_stk_otp_data) ##bytes 형태
opt = opt.content.decode('utf-8') #### byte to string



In [4]:
down_stk_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
down_stk_data = {
    'code' : opt
}
headers = {
    "Referer" : gen_stk_otp_url  #### referer 추가 -> bot으로 인식 안하고 데이터를 제공
}
down = requests.post(down_stk_url, data = down_stk_data, headers = headers )

In [5]:
down_stk_data = down.content.decode('EUC-KR')  #### 다운받은 파일 한글로 디코딩
down_stk_data = down_stk_data.split("\n")    #### \n으로 구분되어 있는거 자르기 -> 한 번 더 ,로 나눠줘야함 -> 가운데 , 포함되어 있는 내용이 있음 ㅠ

#### down_data to csv file -> 어짜피 종목코드랑 종목명만 필요! -> sector는 따로 가져 올 거임
with open('down_stk.csv', 'w', newline = '') as file:
    writer = csv.writer(file)
    for i in range(len(down_stk_data)):
        x = down_stk_data[i].split(",")
        writer.writerow([x[0],x[1]])

In [6]:
pd.read_csv("down_stk.csv").head()

,종목코드,종목명
0,"""095570""","""AJ네트웍스"""
1,"""006840""","""AK홀딩스"""
2,"""027410""","""BGF"""
3,"""282330""","""BGF리테일"""
4,"""138930""","""BNK금융지주"""


------
# 업종분류 현황 크롤링 - 코스닥 -> 추후 코스피랑 한번에 할 수 있도록 짜야함

In [8]:
gen_ksq_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
gen_ksq_otp_data = {
    'mktId' : 'KSQ',   #### 코스피 = stk, 코스닥 = ksq -> option으로 넣으면 될듯
    'trdDd' : '20210312', #### 날짜 -> 추후 영업일로 사용! -> option으로 넣으면 될듯
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : "dbms/MDC/STAT/standard/MDCSTAT03901"
}

opt = requests.post(gen_ksq_otp_url, data = gen_ksq_otp_data) ##bytes 형태
opt = opt.content.decode('utf-8') #### byte to string

down_ksq_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
down_ksq_data = {
    'code' : opt
}
headers = {
    "Referer" : gen_ksq_otp_url  #### referer 추가 -> bot으로 인식 안하고 데이터를 제공
}
down = requests.post(down_ksq_url, data = down_ksq_data, headers = headers )

down_ksq_data = down.content.decode('EUC-KR')  #### 다운받은 파일 한글로 디코딩
down_ksq_data = down_ksq_data.split("\n")    #### \n으로 구분되어 있는거 자르기 -> 한 번 더 ,로 나눠줘야함 -> 가운데 , 포함되어 있는 내용이 있음 ㅠ

#### down_data to csv file -> 어짜피 종목코드랑 종목명만 필요! -> sector는 따로 가져 올 거임
with open('down_ksq.csv', 'w', newline = '') as file:
    writer = csv.writer(file)
    for i in range(len(down_ksq_data)):
        x = down_ksq_data[i].split(",")
        writer.writerow([x[0],x[1]])

In [9]:
pd.read_csv("down_ksq.csv").head()

,종목코드,종목명
0,"""060310""","""3S"""
1,"""054620""","""APS홀딩스"""
2,"""265520""","""AP시스템"""
3,"""211270""","""AP위성"""
4,"""035760""","""CJ ENM"""


------
# 코스피, 코스닥 종목코드 합치기 

In [21]:
stk = pd.read_csv("down_stk.csv") # 코스피
ksq = pd.read_csv("down_ksq.csv") # 코스닥

sec = pd.concat([stk, ksq])
sec.to_csv('down_sec.csv', sep = ',', index = False)

In [26]:
pd.read_csv("down_sec.csv").head()

,종목코드,종목명
0,"""095570""","""AJ네트웍스"""
1,"""006840""","""AK홀딩스"""
2,"""027410""","""BGF"""
3,"""282330""","""BGF리테일"""
4,"""138930""","""BNK금융지주"""


-------
# 개별종목 지표 크롤링 -> 업종분류 현황이랑 합칠 수 있는 부분은 합칠 수 있을 듯
## 아니면 if 문으로 하던지 그냥 따로 쓰던지!

In [328]:
gen_ind_otp_url = "http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd"
gen_ind_otp_data = {
    'searchType' : '1',
    'mktId' : 'ALL',   #### 코스피 = stk, 코스닥 = ksq
    'trdDd' : '20210312', #### 날짜 -> 추후 영업일로 사용!
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : "dbms/MDC/STAT/standard/MDCSTAT03501"
}

opt = requests.post(gen_ind_otp_url, data = gen_ind_otp_data) ##bytes 형태
opt = opt.content.decode('utf-8') #### byte to string

In [329]:
down_ind_url = "http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd"
down_ind_data = {
    'code' : opt
}
headers = {
    "Referer" : gen_ind_otp_url  #### referer 추가 -> bot으로 인식 안하고 데이터를 제공
}
down = requests.post(down_ind_url, data = down_ind_data, headers = headers )

In [338]:
down_ind_data = down.content.decode('EUC-KR')  #### 다운받은 파일 한글로 디코딩
down_ind_data = down_ind_data.split("\n")    #### \n으로 구분되어 있는거 자르기 -> 한 번 더 ,로 나눠줘야함 -> 가운데 , 포함되어 있는 내용이 있음 ㅠ

#### down_data to csv file -> 어짜피 종목코드랑 종목명만 필요! -> sector는 따로 가져 올 거임
with open('down_ind.csv', 'w', newline = '') as file:
    writer = csv.writer(file)
    for i in range(len(down_ind_data)):
        x = down_ind_data[i].split(",")
        writer.writerow(x)

In [339]:
pd.read_csv("down_ind.csv").head()

,종목코드,종목명,종가,대비,등락률,EPS,PER,BPS,PBR,주당배당금,배당수익률
0,"""060310""","""3S ""","""2400""","""5""","""0.21""",NaN,NaN,"""745""","""3.22""","""0""","""0.00"""
1,"""095570""","""AJ네트웍스 ""","""4000""","""30""","""0.76""","""982""","""4.07""","""6802""","""0.59""","""300""","""7.50"""
2,"""006840""","""AK홀딩스 ""","""28700""","""50""","""0.17""","""2168""","""13.24""","""62448""","""0.46""","""750""","""2.61"""
3,"""054620""","""APS홀딩스 ""","""7710""","""150""","""1.98""",NaN,NaN,"""10530""","""0.73""","""0""","""0.00"""
4,"""265520""","""AP시스템 ""","""29000""","""550""","""1.93""","""671""","""43.22""","""7468""","""3.88""","""50""","""0.17"""


------
# 최근 영업일 기준 데이터 받기

In [320]:
get_date_url = "https://finance.naver.com/sise/sise_deposit.nhn"

get_date = requests.get(get_date_url)

#### 진짜 삽질 개오졌네;;; 얼마나 소비한거야 ㅠ 
from lxml import html
get_date = html.fromstring(get_date.content)
date = get_date.xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/div[1]/div/ul[2]/li/span')
date = date[0].xpath('/html/body/div[3]/div[1]/div[2]/div[1]/div[2]/div[1]/div/ul[2]/li/span/text()')

import re

date = re.findall('[0-9]+\.+[0-9]+\.+[0-9]+', date[0])  #### yyyy.mm.dd 형태를 추출하기 위해 정규표현식 사용
biz_date = date[0].replace(".", "") #### yyyy.mm.dd -> yyyymmdd 변환

------
# 거래소 데이터 정리하기

In [118]:
sec = pd.read_csv("down_sec.csv")
ind = pd.read_csv("down_ind.csv")

In [119]:
ind.columns

Index(['종목코드', '종목명', '종가', '대비', '등락률', 'EPS', 'PER', 'BPS', 'PBR', '주당배당금',
       '배당수익률'],
      dtype='object')

In [120]:
sec.columns

Index(['종목코드', '종목명'], dtype='object')

In [121]:
ind['종목명'] = ind['종목명'].str.replace(" ", "") #### 종목명 제일 끝에 공백이 있음 -> 종목명에 공백도 제거 되버림
sec['종목명'] = sec['종목명'].str.replace(" ", "") #### 종목명 중간에 공백을 똑같이 제거 

In [122]:
diff = set(sec['종목명']) - set(ind['종목명']) #### 공통으로 포함되지 않은 종목

In [124]:
KOR_ticker = ind.merge(sec) #### ind를기준으로 공통인 애들로 묶어줌
KOR_ticker.shape

(2367, 11)

In [114]:
KOR_ticker['종목명'][KOR_ticker['종목명'].str.find('스팩') >= 0] #### 스팩주

29        "DB금융스팩8호"
68      "IBKS제12호스팩"
69      "IBKS제13호스팩"
70      "IBKS제14호스팩"
71      "IBKS제15호스팩"
183         "SK4호스팩"
184         "SK5호스팩"
185         "SK6호스팩"
279        "교보10호스팩"
280         "교보9호스팩"
442     "대신밸런스제7호스팩"
443     "대신밸런스제8호스팩"
444     "대신밸런스제9호스팩"
745     "미래에셋대우스팩5호"
746     "미래에셋대우스팩3호"
747     "미래에셋대우스팩4호"
863         "삼성스팩2호"
919      "상상인이안1호스팩"
920     "상상인이안제2호스팩"
1091        "신영스팩5호"
1092        "신영스팩6호"
1106       "신한제6호스팩"
1107       "신한제7호스팩"
1309    "에이치엠씨제4호스팩"
1310    "에이치엠씨제5호스팩"
1352     "엔에이치스팩13호"
1353     "엔에이치스팩16호"
1354     "엔에이치스팩17호"
1355     "엔에이치스팩18호"
1545      "유안타제3호스팩"
1546      "유안타제4호스팩"
1547      "유안타제5호스팩"
1548      "유안타제6호스팩"
1549      "유안타제7호스팩"
1562        "유진스팩4호"
1563        "유진스팩5호"
1600      "이베스트스팩5호"
1601    "이베스트이안스팩1호"
1836      "케이비17호스팩"
1838     "케이비제18호스팩"
1839     "케이비제19호스팩"
1840     "케이비제20호스팩"
1951       "키움제5호스팩"
2109     "하나금융14호스팩"
2110     "하나금융15호스팩"
2111     "하나금융16호스팩"
2112     "하나금융17호스팩"
2116     "하나머

In [135]:
KOR_ticker = KOR_ticker[KOR_ticker['종목명'].str.find("스팩") == -1] #### 스팩주 제거

In [163]:
KOR_ticker['종목명'][KOR_ticker['종목코드'].str[-2] != "0"] #### 우선주 목록 -> 가장 끝은 ""이라서 그 다음 문자열을 찾아줘야함

10          "BYC우"
14      "CJ4우(전환)"
17       "CJ씨푸드1우"
18           "CJ우"
20       "CJ제일제당우"
           ...    
2302      "현대차3우B"
2303        "현대차우"
2317       "호텔신라우"
2362     "흥국화재2우B"
2363       "흥국화재우"
Name: 종목명, Length: 121, dtype: object

In [164]:
KOR_ticker = KOR_ticker[KOR_ticker['종목코드'].str[-2] == "0"] #### 우선주 제거

In [166]:
KOR_ticker.to_csv("KOR_ticker.csv", index=False)

-----
# WICS 기준 섹터 정보 크롤링

In [219]:
import time

sector_code = ['G25', 'G35', 'G50', 'G40', 'G10',
              'G20', 'G55', 'G30', 'G15', 'G45',]
data_sector = pd.DataFrame()

for code in sector_code:
    url = "http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20210316&sec_cd=" + code
    data = requests.get(url)
    data_df = pd.DataFrame(data.json()['list']) #### 받아온 결과를 json으로 출력 -> dict로 출력되는 데 key가 'list'가 해당 정보
    data_sector = pd.concat([data_sector, data_df])
    time.sleep(1) #### 1초간 휴식 -> R에서 Sys.sleep(1)과 같은역할

In [227]:
data_sector.index = range(len(data_sector)) #### index를 편하게 숫자순서대로 변경

In [229]:
data_sector.to_csv("KOR_sector.csv", index=False)

-----
# Ch6
# 개별 종목 수정주가 크롤링 - 더 효율적인 방법 찾아야함 - 너무 무식하게 짰다....

In [68]:
url = "https://fchart.stock.naver.com/siseJson.nhn?symbol=005930&requestType=1&startTime=20190608&endTime=20200622&timeframe=day"

In [69]:
url_data = requests.get(url)

In [70]:
url_data = url_data.content.decode("utf8").replace("\t", "").replace("\n", "") #### \t, \n을 공백으로 대체

In [71]:
url_data = url_data.replace("[","")
url_data = url_data.replace(" ","")
url_data = url_data.split("]")    #### 최대한 안에 결과 값들만 뽑아내기 위함

In [72]:
#### 맨 앞의 ','를 제거해주기 위함

df = []
df.append(url_data[0])  #### 가장 첫줄 먼저 포함
for i in range(1, len(url_data)):
    x = url_data[i][1:]
    df.append(x)


In [73]:
df = df[:-2] #### 가장 마지막 2개가 빈 칸이므로 제거

In [74]:
#### 최대한 dataframe에 맞춰서 만듦 - expand는 시리즈일 때만 있는 옵션인듯 - split 결과를 서로 다른 열에 넣어줌
#### 첫번째 row를 행 이름으로 쓰기위해서 이 방법이 진짜 이게 최선이라고...? ㅋ
df = pd.DataFrame(df)[0].str.split(",", expand=True)
df.columns = df.iloc[0,:]
df = df.iloc[1:,:]

In [75]:
df["'날짜'"] = pd.to_datetime(df["'날짜'"], format='"%Y%m%d"') #### 날짜형태로 변환 - columns 이름이랑 날짜 값들이 모두 ""를 값으로 포함

In [76]:
df.set_index("'날짜'", inplace = True) #### 날짜를 인덱스로 변환

In [82]:
df_price_volume = df[["'종가'", "'거래량'"]] #### 종가, 거래량 추출

In [90]:
df_price_volume.to_csv("KOR_price_volume.csv") #### csv로 저장

-------
# 전 종목 주가 크롤링 - 위에서 짠 개인 종목을 ticker를 불러와서 ticker정보를 통해 url 변경
# csv 파일이 많아지므로 새로운 폴더를 만드는 과정이 가장 먼저 필요